# Airport Connections
[link](https://www.algoexpert.io/questions/Airport%20Connections)

## My Solution

In [ ]:
# O(a * (a + r)) time | O(a + r) space
def airportConnections(airports, routes, startingAirport):
    # Write your code here.
    airportsGraph = createGraph(airports, routes)
    for airportName in airports:
        traverseNodes(airportsGraph, airportName)
        
    l = [airportsGraph[airportName] for airportName in airportsGraph if airportName != startingAirport]
    l.sort(key=lambda x: x.score, reverse=True)
    l = [airportsGraph[startingAirport]] + l
    
    count = -1
    
    for airportNode in l:
        if airportNode.reached == True:
            continue
        count += 1
        reaches = traverseNodes(airportsGraph, airportNode.name)
        for airportName in reaches:
            airportsGraph[airportName].reached = True

    return count

def createGraph(nodes, edges):
    graph = {a: Node(a) for a in nodes}
    for r in edges:
        graph[r[0]].tos.append(r[1])
    return graph

def traverseNodes(graph, startNodeName):
    visited = {startNodeName: True}
    stack = [startNodeName]
    while len(stack) != 0:
        curName = stack.pop()
        visited[curName] = True
        curNode = graph[curName]
        for nextName in curNode.tos:
            if nextName not in visited:
                stack.append(nextName)
    graph[startNodeName].score = len(visited)
    return visited

class Node:
    def __init__(self, name):
        self.name = name
        self.score = 0
        self.tos = []
        self.reached = False

## Expert Solution

In [ ]:
# O(a * (a + r) + a + r + alog(a)) time | O(a + r) space - where a is the number of airports and r is the number of routes 
def airportConnections(airports, routes, startingAirport):
    airportGraph = createAirportGraph(airports, routes)
    unreachableAirportNodes =  getUnreachableAirportNodes(airportGraph, airports, startingAirport)
    markUnreachableConnections(airportGraph, unreachableAirportNodes)
    return getMinNumberOfNewConnections(airportGraph, unreachableAirportNodes)

# O(a + r) time | O(a + r) space
def createAirportGraph(airports, routes):
    airportGraph = {}
    for airport in airports:
        airportGraph[airport] = AirportNode(airport)
    for route in routes:
        airport, connection = route
        airportGraph[airport].connections.append(connection)
    return airportGraph

# O(a + r) time | O(a) space
def getUnreachableAirportNodes(airportGraph, airports, startingAirport):
    visitedAirports = {}
    depthFirstTraverseAirports(airportGraph, startingAirport, visitedAirports)
    
    unreachableAirportNodes = []
    for airport in airports:
        if airport in visitedAirports:
            continue
        airportNode = airportGraph[airport]
        airportNode.isReachable = False
        unreachableAirportNodes.append(airportNode)
    return unreachableAirportNodes
    
    
def depthFirstTraverseAirports(airportGraph, airport, visitedAirports):
    if airport in visitedAirports:
        return
    visitedAirports[airport] = True
    connections = airportGraph[airport].connections
    for connection in connections:
        depthFirstTraverseAirports(airportGraph,  connection, visitedAirports)


# O(a * (a + r)) time | O(a) space
def markUnreachableConnections(airportGraph, unreachableAirportNodes):
    for airportNode in unreachableAirportNodes:
        airport = airportNode.airport
        unreachableConnections = []
        depthFirstAddUnreachableConnections(airportGraph, airport, unreachableConnections, {})
        airportNode.unreachableConnections = unreachableConnections

def depthFirstAddUnreachableConnections(airportGraph, airport, unreachableConnections, visitedAirports):
    if airportGraph[airport].isReachable:
        return
    if airport in visitedAirports:
        return
    visitedAirports[airport] = True
    unreachableConnections.append(airport)
    connections = airportGraph[airport].connections
    for connection in connections:
        depthFirstAddUnreachableConnections(airportGraph, connection, unreachableConnections, visitedAirports)
        
# O(alog(a) + a + r) time | O(1) space
def getMinNumberOfNewConnections(airportGraph, unreachableAirportNodes):
    unreachableAirportNodes.sort(key=lambda airport: len(airport.unreachableConnections), reverse=True)
    
    numberOfNewConnections = 0
    for airportNode in unreachableAirportNodes:
        if airportNode.isReachable:
            continue
        numberOfNewConnections += 1
        for connection in airportNode.unreachableConnections:
            airportGraph[connection].isReachable = True
    return numberOfNewConnections

class AirportNode:
    def __init__(self, airport):
        self.airport = airport
        self.connections = []
        self.isReachable = True
        self.unreachableConnections = []

## Thoughts
### Argue
in expert solution, `markUnreachableConnections` function should need O(a^2) space.